# **CHAPTER 1. Pangenome analysis**

Import all the modules needed

In [1]:
import os
import re
import csv
import pandas as pd
from Bio import Entrez, SeqIO

In this study `PanACoTA` software will be used for Pangenome analysis

`PanACoTA` uses `Prokka` for homologous genomes annotations, which is not good for fungal genomes

So, we will use already annotated mitochondrial genomes from `RefSeq` database!

First, create a directory to store all the data

In [3]:
%%bash

mkdir pangenome/
mkdir pangenome/data/
mkdir pangenome/Annotation/
mkdir pangenome/Annotation/Genes/
mkdir pangenome/Annotation/Proteins/
mkdir pangenome/Annotation/Proteins_classic/

Then we will need the list of accession numbers of fungal complete mitochondrial genomes

In [2]:
! esearch -db nucleotide \
    -query '("Leotiomycetes"[Organism] OR Leotiomycetes[All Fields]) AND srcdb_refseq[PROP] AND (fungi[filter] AND mitochondrion[filter])' \
    | efetch -format acc > pangenome/data/accession_numbers.txt

This function will download us everything

In [3]:
def get_sequences(email, file_path, output_dir, format, extension):
    Entrez.email = email

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Read accession numbers from file
    with open(file_path, "r") as file:
        accession_numbers = file.read().split()

    def download_sequence(accession):
        """Fetches a single sequence from NCBI and saves it as a FASTA file."""
        try:
            handle = Entrez.efetch(
                db="nucleotide", id=accession, rettype=format, retmode="text"
            )
            records = list(SeqIO.parse(handle, "fasta"))  # Use parse() instead of read()
            handle.close()

            if records:
                output_path = os.path.join(output_dir, f"{accession.split('.')[0]}.{extension}")
                SeqIO.write(records, output_path, "fasta")
                print(f"Downloaded: {accession}")
            else:
                print(f"No CDS found for {accession}")

        except Exception as e:
            print(f"Failed to download {accession}: {e}")

    # Download sequences for each accession number
    for accession in accession_numbers:
        download_sequence(accession)

    print("All downloads completed.")

In [4]:
email = "ivpopov@donstu.ru"
accession_numbers = "pangenome/data/accession_numbers.txt"

Download genomes

In [ ]:
get_sequences(email,
              accession_numbers,
              "pangenome/Annotation/Genes",
              format = "fasta_cds_na",
              extension = "gen")

Download proteomes

In [ ]:
get_sequences(email,
              accession_numbers,
              "pangenome/Annotation/Proteins_classic",
              format = "fasta_cds_aa",
              extension = "prt")

Now, we need to rename downloaded mitochondrial proteomes to make them face the requirements of `PanACoTA` input

In [7]:
%%bash

for file in pangenome/Annotation/Proteins_classic/*.prt; do 
    awk '{
        if ($0 ~ /^>/) {
            gsub(/^>lcl\|/,">");                        # Remove "lcl|"
            match($0, /^>([^.]+)\.[0-9]+_prot_/, g);    # Extract genome name (e.g., NC_015789)
            match($0, /_prot_YP_([0-9]+)/, id);         # Extract numeric protein ID (e.g., 004733034)
            if (g[1] != "" && id[1] != "") {
                print ">" g[1] "_" id[1];               # Format as genomeName_numericID
            } else {
                print ">" $2;  # Fallback if parsing fails
            }
        } else {
            print;
        }
    }' "$file" > pangenome/Annotation/Proteins/$(basename "$file")
done

Now we apply the renaming from proteomes to genomes!

In [ ]:
# Define directories
proteins_dir = "pangenome/Annotation/Proteins"
genes_dir = "pangenome/Annotation/Genes"

# Ensure Genes directory exists
if not os.path.exists(genes_dir):
    print("Genes directory does not exist.")
    exit(1)

# Function to extract FASTA sequences
def read_fasta(file_path):
    sequences = []
    with open(file_path, "r") as f:
        seq = []
        header = None
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                if header:
                    sequences.append((header, "\n".join(seq)))
                header = line  # Store new header
                seq = []
            else:
                seq.append(line)
        if header:
            sequences.append((header, "\n".join(seq)))  # Append last sequence
    return sequences

# Function to write updated FASTA sequences
def write_fasta(file_path, sequences):
    with open(file_path, "w") as f:
        for header, seq in sequences:
            f.write(f"{header}\n{seq}\n")

# Process all .prt files in Proteins directory
for prt_file in os.listdir(proteins_dir):
    if prt_file.endswith(".prt"):
        # Get corresponding .gen file
        base_name = os.path.splitext(prt_file)[0]  # Remove .prt extension
        gen_file = f"{base_name}.gen"
        
        prt_path = os.path.join(proteins_dir, prt_file)
        gen_path = os.path.join(genes_dir, gen_file)

        # Check if corresponding .gen file exists
        if not os.path.exists(gen_path):
            print(f"Skipping {gen_file} (not found in Genes directory)")
            continue

        # Read sequences from .prt and .gen files
        prt_seqs = read_fasta(prt_path)
        gen_seqs = read_fasta(gen_path)

        # Ensure both files have the same number of sequences
        if len(prt_seqs) != len(gen_seqs):
            print(f"Skipping {gen_file} (mismatch: {len(prt_seqs)} protein seqs vs {len(gen_seqs)} gene seqs)")
            continue

        # Replace headers in .gen file
        updated_gen_seqs = [(prt_seqs[i][0], gen_seqs[i][1]) for i in range(len(gen_seqs))]

        # Write updated .gen file
        write_fasta(gen_path, updated_gen_seqs)
        print(f"Updated {gen_file} with new headers from {prt_file}")

print("Processing complete.")

Now create a list file with the proteomes to build the pangenome

In [9]:
! ls pangenome/Annotation/Proteins/*.prt | sed 's|pangenome/Annotation/Proteins/||' | sed 's/\.prt$//' >\
    pangenome/Annotation/LSTINFO-.lst

Also, we must create a merged proteomes dataset

In [10]:
! cat pangenome/Annotation/Proteins/* > pangenome/Annotation/Proteins/LeMy.All.prt

Good! Now let's construct a pangenome with the proteins identity setting = `0.9` (90%).

In [8]:
! PanACoTA pangenome -l pangenome/Annotation/LSTINFO-.lst -n LeMy -d pangenome/Annotation/Proteins/ -o pangenome/Pangenome -i 0.9

  * [2025-03-17 21:18:57] : INFO  PanACoTA version 1.4.0
  * [2025-03-17 21:18:57] : INFO  Command used
 	 > PanACoTA pangenome -l pangenome/Annotation/LSTINFO-.lst -n LeMy -d pangenome/Annotation/Proteins/ -o pangenome/Pangenome -i 0.9
  * [2025-03-17 21:18:57] : INFO  Will run MMseqs2 with:
	- minimum sequence identity = 90.0%
	- cluster mode 1
  * [2025-03-17 21:18:57] : INFO  Reading and getting information from pangenome file
  * [2025-03-17 21:18:57] : INFO  Retrieving information from pan families
  * [2025-03-17 21:18:57] : INFO  Generating qualitative and quantitative matrix, and summary file
  * [2025-03-17 21:18:57] : INFO  DONE


For now please proceed to the `02_pangenome_visualization.R` and run the analysis there. Then come back!

So, there are 4 genes that are presented in more than 17 genomes out of 24! 17 is more than 16, so let's calculate the percentage of genomes sharing these 4 genes!

In [2]:
16 * 100 / 24

66.66666666666667

Perfect! Now run `PanACoTA`'s `corepers` module to extract those genes!

In [9]:
! PanACoTA corepers -p pangenome/Pangenome/PanGenome-LeMy.All.prt-clust-0.9-mode1.lst -o pangenome/Coregenome -t 0.66

  * [2025-03-17 21:19:07] : INFO  PanACoTA version 1.4.0
  * [2025-03-17 21:19:07] : INFO  Command used
 	 > PanACoTA corepers -p pangenome/Pangenome/PanGenome-LeMy.All.prt-clust-0.9-mode1.lst -o pangenome/Coregenome -t 0.66
  * [2025-03-17 21:19:07] : INFO  Will generate a Persistent genome with member(s) in at least 66.0% of all genomes in each family.
To be considered as persistent, a family must contain exactly 1 member in at least 66.0% of all genomes. The other genomes are absent from the family.
  * [2025-03-17 21:19:07] : INFO  Retrieving info from binary file
  * [2025-03-17 21:19:07] : INFO  Generating Persistent genome of a dataset containing 24 genomes
  * [2025-03-17 21:19:07] : INFO  The persistent genome contains 4 families, each one having exactly 1 member from at least 66.0% of the 24 different genomes (that is 16 genomes). The other genomes are absent from the family.
  * [2025-03-17 21:19:07] : INFO  Persistent genome step done.


Process `pangenome/Annotation/LSTINFO-.lst` file to leave there only those 66% of genomes that have 4 commonly shared genes!

In [2]:
def process_files(input_txt, input_tsv, output_tsv):
    """
    Processes two input files and creates a filtered TSV file.

    Args:
        input_txt (str): Path to the input TXT file.
        input_tsv (str): Path to the input TSV file (no header, one column).
        output_tsv (str): Path to the output TSV file.
    """

    def trim_identifier(identifier):
        """
        Trims the identifier to retain only the part before the 2nd underscore.

        Args:
            identifier (str): The full identifier string.

        Returns:
            str: The trimmed identifier.
        """
        return "_".join(identifier.split("_")[:2])

    # Extract trimmed "gembase_name" values from the first two rows of the TXT file
    with open(input_txt, "r") as txt_file:
        lines = txt_file.readlines()
        if len(lines) < 2:
            raise ValueError("The TXT file must have at least two rows.")
        first_row = lines[0].strip().split()
        second_row = lines[1].strip().split()
        gembase_names = {trim_identifier(item) for item in first_row + second_row}

    # Read the TSV file (single column, no header) and filter rows
    with open(input_tsv, "r") as tsv_file:
        reader = csv.reader(tsv_file, delimiter="\t")
        filtered_rows = [
            row for row in reader if trim_identifier(row[0]) in gembase_names
        ]

    # Write the filtered rows to a new TSV file (single column, no header)
    with open(output_tsv, "w", newline="") as out_file:
        writer = csv.writer(out_file, delimiter="\t")
        writer.writerows(filtered_rows)

    print(f"Filtered file created: {output_tsv}")

In [3]:
input_txt = "pangenome/Coregenome/PersGenome_PanGenome-LeMy.All.prt-clust-0.9-mode1.lst-all_0.66.lst"
input_tsv = "pangenome/Annotation/LSTINFO-.lst"
output_tsv = "pangenome/Annotation/fLSTINFO-.lst"

process_files(input_txt, input_tsv, output_tsv)

Filtered file created: pangenome/Annotation/fLSTINFO-.lst


Lifehack: instead of running `MAFFT` by ourselves, we can still run `PanACoTA`!

In [10]:
! PanACoTA align -c pangenome/Coregenome/PersGenome_PanGenome-LeMy.All.prt-clust-0.9-mode1.lst-all_0.66.lst\
    -l pangenome/Annotation/fLSTINFO-.lst -n LeMy -d pangenome/Annotation/ -o pangenome/Alignment

  * [2025-03-17 21:19:11] : INFO  PanACoTA version 1.4.0
  * [2025-03-17 21:19:11] : INFO  Command used
 	 > PanACoTA align -c pangenome/Coregenome/PersGenome_PanGenome-LeMy.All.prt-clust-0.9-mode1.lst-all_0.66.lst -l pangenome/Annotation/fLSTINFO-.lst -n LeMy -d pangenome/Annotation/ -o pangenome/Alignment
  * [2025-03-17 21:19:11] : INFO  Found 23 genomes.
  * [2025-03-17 21:19:11] : INFO  Reading PersGenome and constructing lists of missing genomes in each family.
  * [2025-03-17 21:19:11] : INFO  Getting all persistent proteins and classify by strain.
  * [2025-03-17 21:19:11] : INFO  All extraction files already existing (see detailed log for more information)
  * [2025-03-17 21:19:11] : INFO  Starting alignment of all families: protein alignment, back-translation to nucleotides, and add missing genomes in the family
Alignment:                                0/4 (  0%) - Elapsed Time: 0:00:00 -   * [2025-03-17 21:19:11] : INFO  nucl alignments already concatenated
  * [2025-03-17 

Now it is time to work with MSAs a little bit<br>
First, create a directory where to store them

In [2]:
! mkdir pangenome/Alignment/MSAs/

Let's take a look at the current state of MSAs

In [3]:
! head -1 pangenome/Alignment/Align-LeMy/LeMy-mafft-align.6.aln

>NC_015789_004733052


Well, that's good. We just need to delete everything after the 2nd `_` including that 2nd `_`. Also, it will be good to rename files to know what gene is in that MSA file

In [4]:
def process_msa(input_fasta, output_fasta):
    """
    Processes a multiple sequence alignment (MSA) file, renaming sequence headers
    by removing everything after the first underscore in each header.

    Args:
        input_fasta (str): Path to the input FASTA file.
        output_fasta (str): Path to save the processed FASTA file.
    """

    def trim_header(header):
        """Removes everything after the 2nd underscore in a sequence header."""
        return "_".join(header.split("_")[:2])

    # Read and process the FASTA file
    with open(input_fasta, "r") as infile, open(output_fasta, "w") as outfile:
        for line in infile:
            if line.startswith(">"):  # Header line
                new_header = f">{trim_header(line[1:].strip())}\n"
                outfile.write(new_header)
            else:
                outfile.write(line)  # Write sequence lines unchanged

    print(f"Processed MSA saved to {output_fasta}")

In [5]:
%%bash

head -1 pangenome/Alignment/Align-LeMy/LeMy-mafft-align.6.aln
id=$(head -1 pangenome/Alignment/Align-LeMy/LeMy-mafft-align.6.aln | cut -d'_' -f3)
grep "$id" pangenome/Annotation/Proteins_classic/NC_015789.prt

>NC_015789_004733052
>lcl|NC_015789.1_prot_YP_004733052.1_19 [gene=ND4L] [locus_tag=PhsufM_p19] [db_xref=GeneID:10963959] [protein=NADH dehydrogenase subunit 4L] [protein_id=YP_004733052.1] [location=37495..37764] [gbkey=CDS]


In [6]:
fasta_file = 'pangenome/Alignment/Align-LeMy/LeMy-mafft-align.6.aln'
output_fasta_file = 'pangenome/Alignment/MSAs/nad4l.aln'

process_msa(fasta_file, output_fasta_file)

Processed MSA saved to pangenome/Alignment/MSAs/nad4l.aln


In [7]:
%%bash

head -1 pangenome/Alignment/Align-LeMy/LeMy-mafft-align.32.aln
id=$(head -1 pangenome/Alignment/Align-LeMy/LeMy-mafft-align.32.aln | cut -d'_' -f3)
grep "$id" pangenome/Annotation/Proteins_classic/NC_015789.prt

>NC_015789_004733049
>lcl|NC_015789.1_prot_YP_004733049.1_16 [gene=COX2] [locus_tag=PhsufM_p16] [db_xref=GeneID:10963961] [protein=cytochrome c oxidase subunit II] [protein_id=YP_004733049.1] [location=31173..31922] [gbkey=CDS]


In [ ]:
fasta_file = 'pangenome/Alignment/Align-LeMy/LeMy-mafft-align.32.aln'
output_fasta_file = 'pangenome/Alignment/MSAs/cox2.aln'

process_msa(fasta_file, output_fasta_file)

Processed MSA saved to pangenome/Alignment/MSAs/cox2.aln


In [9]:
%%bash

head -1 pangenome/Alignment/Align-LeMy/LeMy-mafft-align.130.aln
id=$(head -1 pangenome/Alignment/Align-LeMy/LeMy-mafft-align.130.aln | cut -d'_' -f3)
grep "$id" pangenome/Annotation/Proteins_classic/NC_015789.prt

>NC_015789_004733054
>lcl|NC_015789.1_prot_YP_004733054.1_21 [gene=CYTB] [locus_tag=PhsufM_p21] [db_xref=GeneID:10963966] [protein=cytochrome b] [protein_id=YP_004733054.1] [location=41872..43038] [gbkey=CDS]


In [ ]:
fasta_file = 'pangenome/Alignment/Align-LeMy/LeMy-mafft-align.130.aln'
output_fasta_file = 'pangenome/Alignment/MSAs/cob.aln'

process_msa(fasta_file, output_fasta_file)

Processed MSA saved to pangenome/Alignment/MSAs/cob.aln


In [11]:
%%bash

head -1 pangenome/Alignment/Align-LeMy/LeMy-mafft-align.565.aln
id=$(head -1 pangenome/Alignment/Align-LeMy/LeMy-mafft-align.565.aln | cut -d'_' -f3)
grep "$id" pangenome/Annotation/Proteins_classic/NC_015789.prt

>NC_015789_004733034
>lcl|NC_015789.1_prot_YP_004733034.1_1 [gene=COX1] [locus_tag=PhsufM_p01] [db_xref=GeneID:10963948] [protein=cytochrome c oxidase subunit I] [protein_id=YP_004733034.1] [location=481..2202] [gbkey=CDS]


In [12]:
fasta_file = 'pangenome/Alignment/Align-LeMy/LeMy-mafft-align.565.aln'
output_fasta_file = 'pangenome/Alignment/MSAs/cox1.aln'

process_msa(fasta_file, output_fasta_file)

Processed MSA saved to pangenome/Alignment/MSAs/cox1.aln


When the MSAs are ready it is time to trim them all!!!

First, create a directory to store trimmed MSAs

In [13]:
! mkdir pangenome/Alignment/trimmed_MSAs/

Then run a bash loop with `trimAl` in it on them

In [ ]:
%%bash

for msa in pangenome/Alignment/MSAs/*.aln
do trimal -in $msa -out pangenome/Alignment/trimmed_MSAs/$(basename "$msa" .aln)_trim.fa -automated1
done

We've got the trimmed MSAs! Now what? `MODELFINDER`!

First, create a directory to store `ModelFinder` log files

In [13]:
! mkdir pangenome/model-finder/

Then run a bash loop with `ModelFinder` in it on trimmed MSAs!

In [14]:
%%bash

for msa in pangenome/Alignment/trimmed_MSAs/*_trim.fa
do 
    iqtree2 -m MFP -s $msa --prefix pangenome/model-finder/$(basename "$msa" _trim.fa) -T AUTO
done

IQ-TREE multicore version 2.4.0 for MacOS Intel 64-bit built Feb 12 2025
Developed by Bui Quang Minh, Nguyen Lam Tung, Olga Chernomor, Heiko Schmidt,
Dominik Schrempf, Michael Woodhams, Ly Trong Nhan, Thomas Wong

Host:    Ilas-Mac-mini.local (SSE4.2, 16 GB RAM)
Command: iqtree2 -m MFP -s pangenome/Alignment/trimmed_MSAs/cob_trim.fa --prefix pangenome/model-finder/cob -T AUTO
Seed:    424964 (Using SPRNG - Scalable Parallel Random Number Generator)
Time:    Mon Mar 17 21:19:48 2025
Kernel:  SSE2 - auto-detect threads (10 CPU cores detected)

Reading alignment file pangenome/Alignment/trimmed_MSAs/cob_trim.fa ... Fasta format detected
Reading fasta file: done in 0.000603914 secs using 92.56% CPU
Alignment most likely contains protein sequences
Alignment has 17 sequences with 358 columns, 91 distinct patterns
45 parsimony-informative, 28 singleton sites, 285 constant sites
           Gap/Ambiguity  Composition  p-value
Analyzing sequences: done in 1.71661e-05 secs using 570.9% CPU
   1  

Get the best fit model for `nad4l`

In [15]:
! head -42 pangenome/model-finder/nad4l.iqtree | tail -6

Best-fit model according to BIC: mtZOA

List of models sorted by BIC scores: 

Model                  LogL         AIC      w-AIC        AICc     w-AICc         BIC      w-BIC
mtZOA              -271.267     592.535 +    0.136     618.025 +    0.777     651.130 +    0.755


Get the best fit model for `cox2`

In [16]:
! head -42 pangenome/model-finder/cox2.iqtree | tail -6

Best-fit model according to BIC: Q.plant+G4

List of models sorted by BIC scores: 

Model                  LogL         AIC      w-AIC        AICc     w-AICc         BIC      w-BIC
Q.plant+G4         -967.262    2002.524 +    0.346    2015.389 +    0.383    2117.908 +    0.446


Get the best fit model for `cob`

In [17]:
! head -42 pangenome/model-finder/cob.iqtree | tail -6

Best-fit model according to BIC: mtZOA+G4

List of models sorted by BIC scores: 

Model                  LogL         AIC      w-AIC        AICc     w-AICc         BIC      w-BIC
mtZOA+G4          -1726.355    3512.710 +    0.339    3518.398 +    0.383    3629.126 +    0.732


Get the best fit model for `cox1`

In [18]:
! head -42 pangenome/model-finder/cox1.iqtree | tail -6

Best-fit model according to BIC: mtZOA+R2

List of models sorted by BIC scores: 

Model                  LogL         AIC      w-AIC        AICc     w-AICc         BIC      w-BIC
mtZOA+R2          -2329.417    4720.834 +    0.593    4724.770 +    0.731    4853.642 +    0.483


At last... We have best fit models for all of our 4 genes... It's time to launch `IQ-TREE`!

First, create a directory to store the trees

In [9]:
! mkdir pangenome/tree/

Then run a bash loop with `IQ-TREE` in it on trimmed MSAs

In [10]:
%%bash

for msa in pangenome/Alignment/trimmed_MSAs/*_trim.fa
do 
    if [[ "$msa" == "nad4l.fa" ]]; then
        model="mtZOA"
    elif [[ "$msa" == "cox2.fa" ]]; then
        model="Q.plant+G4"
    elif [[ "$msa" == "cob.fa" ]]; then
        model="mtZOA+G4"
    elif [[ "$msa" == "cox1.fa" ]]; then
        model="mtZOA+R2"
    fi

    iqtree2 -s "$msa" -m "$model" -pre pangenome/tree/$(basename "$msa" _trim.fa)_ufb -bb 10000 -nt AUTO
done

IQ-TREE multicore version 2.4.0 for MacOS Intel 64-bit built Feb 12 2025
Developed by Bui Quang Minh, Nguyen Lam Tung, Olga Chernomor, Heiko Schmidt,
Dominik Schrempf, Michael Woodhams, Ly Trong Nhan, Thomas Wong

Host:    Ilas-Mac-mini.local (SSE4.2, 16 GB RAM)
Command: iqtree2 -s pangenome/Alignment/trimmed_MSAs/cob_trim.fa -m  -pre pangenome/tree/cob_ufb -bb 10000 -nt AUTO
Seed:    987020 (Using SPRNG - Scalable Parallel Random Number Generator)
Time:    Mon Mar 17 20:31:29 2025
Kernel:  SSE2 - auto-detect threads (10 CPU cores detected)

Reading alignment file pangenome/Alignment/trimmed_MSAs/cob_trim.fa ... Fasta format detected
Reading fasta file: done in 0.000647783 secs using 62.68% CPU
Alignment most likely contains protein sequences
Alignment has 17 sequences with 358 columns, 91 distinct patterns
45 parsimony-informative, 28 singleton sites, 285 constant sites
           Gap/Ambiguity  Composition  p-value
Analyzing sequences: done in 0.000113964 secs using 343.1% CPU
   1  

The last step is adding `.1` to accession numbers on the tree

In [11]:
def modify_tree_file(input_file, output_file):
    """
    Reads a Newick tree from a file, adds ".1" to all accession numbers, 
    and writes the modified tree to an output file.

    Args:
        input_file (str): Path to the input tree file.
        output_file (str): Path to save the modified tree.
    """

    def add_suffix(match):
        return match.group(0) + ".1"

    # Read tree from file
    with open(input_file, "r") as infile:
        tree_str = infile.read().strip()

    # Modify tree
    modified_tree = re.sub(r'NC_\d+', add_suffix, tree_str)

    # Write modified tree to output file
    with open(output_file, "w") as outfile:
        outfile.write(modified_tree)

    print(f"Modified tree saved to: {output_file}")

In [14]:
tree_files = ["nad4l", "cox2", "cob", "cox1"]

for gene in tree_files:
    input_tree_file = f"pangenome/tree/{gene}_ufb.treefile"
    output_tree_file = f"pangenome/tree/{gene}_ufb_ready.treefile"
    modify_tree_file(input_tree_file, output_tree_file)

Modified tree saved to: pangenome/tree/nad4l_ufb_ready.treefile
Modified tree saved to: pangenome/tree/cox2_ufb_ready.treefile
Modified tree saved to: pangenome/tree/cob_ufb_ready.treefile
Modified tree saved to: pangenome/tree/cox1_ufb_ready.treefile


Actually that's all!<br>
But now we'll fetch metadata on _Leotiomycetes_ fungi from NCBI RefSeq<br>
It will be used to annotate the trees in `ggtree`!

First create a directory where to store metadata

Now fetch metadata!

In [ ]:
! mkdir metadata/

In [ ]:
! Phyloki --fetch_metadata -email ivpopov@donstu.ru -i pangenome/data/accession_numbers.txt -o metadata/raw_metadata.tsv

This function will clean the `Year` column!

In [ ]:
def clean_year(input_file, output_file):
    """
    Clean the 'Year' column in a metadata .tsv file to extract only the last 4 digits.

    Args:
        input_file (str): Path to the input .tsv file.
        output_file (str): Path to save the cleaned .tsv file.
    """
    # Load the .tsv file into a DataFrame
    df = pd.read_csv(input_file, sep="\t")

    # Extract the last 4 digits of the 'Year' column
    df['Year'] = df['Year'].apply(lambda x: str(x)[-4:] if pd.notnull(x) else 'ND')

    # Save the updated DataFrame to a new .tsv file
    df.to_csv(output_file, sep="\t", index=False)

    print(f"The 'Year' column has been cleaned.\nFile saved to {output_file}")

In [ ]:
clean_year('metadata/raw_metadata.tsv',
           'metadata/metadata.tsv')

That's all for pangenome analysis! Please proceed to the `03_phylogenomics.ipynb` for further analysis!